In [99]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import os
import PIL
import tensorflow as tf
import time
import sys
import math

from tensorflow.keras import backend as kb
from resizeimage import resizeimage
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Layer, Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential

from IPython.display import clear_output
from tqdm import tqdm

In [100]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 8)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)


In [101]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)


tmp = list(data_dir.glob('*/'))
classes = []
for i in tmp:
    if os.path.isdir(i):
        classes.append(os.path.basename(i))
print(classes)

Images = []
for class_ in classes:
    list_ = list(data_dir.glob(class_ + '/*'))
    Images.append(list_)
    

numBatches = 100
width = 180
height = 180

inputs_train = np.empty([1, len(classes), width, height, 3])
for i in range(int(numBatches * 0.7)):
    
    input_ = np.empty([1, width, height, 3])
    
    for class_ in range(len(classes)):
        numClass = len(Images[class_])
        choice_idx = np.random.choice(numClass, 1, replace=False)
        choice = [Images[class_][j] for j in choice_idx]
        
        with PIL.Image.open(choice[0]) as img:
            im = resizeimage.resize_cover(img, [width, height])
            im = np.expand_dims(np.array(im), axis=0)
            input_ = np.concatenate((input_, im), axis=0)
    input_ = np.delete(input_, 0, axis=0)
    
    input_ = np.expand_dims(input_, axis=0)
    inputs_train = np.concatenate((inputs_train, input_), axis=0)
    
inputs_train = np.delete(inputs_train, 0, axis=0)




inputs_test = np.empty([1, len(classes), width, height, 3])
for i in range(int(numBatches * 0.3)):
    
    input_ = np.empty([1, width, height, 3])
    
    for class_ in range(len(classes)):
        numClass = len(Images[class_])
        choice_idx = np.random.choice(numClass, 1, replace=False)
        choice = [Images[class_][j] for j in choice_idx]
        
        with PIL.Image.open(choice[0]) as img:
            im = resizeimage.resize_cover(img, [width, height])
            im = np.expand_dims(np.array(im), axis=0)
            input_ = np.concatenate((input_, im), axis=0)
    input_ = np.delete(input_, 0, axis=0)
    
    input_ = np.expand_dims(input_, axis=0)
    inputs_test = np.concatenate((inputs_test, input_), axis=0)
    
inputs_test = np.delete(inputs_test, 0, axis=0)

print(inputs_train.shape)
print(inputs_test.shape)

['dandelion', 'sunflowers', 'daisy', 'roses', 'tulips']
(70, 5, 180, 180, 3)
(30, 5, 180, 180, 3)


In [ ]:
drawFeatureMap(model, idxBatch = 0, class_ = 1, features = [0, 1])

In [126]:
def drawFeatureMap(epoch, model, idxBatch, class_):
    
    numFeatures = model.numFeatures
    inputs = inputs_train[idxBatch,:,:,:,:]
    
    img_shape = inputs[class_,:,:,0].shape
    
    width_ratios = [1]
    for i in range(math.ceil(numFeatures / 3)):
        width_ratios.append(1)
    fig, axs = plt.subplots(inputs.shape[-1], math.ceil(numFeatures / inputs.shape[-1]) + 1, gridspec_kw={'width_ratios': width_ratios})
    
    
    for color in range(3):
        axs[color, 0].imshow(inputs[class_,:,:,color], cmap='gray', vmin=0, vmax=255)
        
    outputs = model(inputs)
    single_output = outputs[class_,:,:,:]
    single_output_abs = tf.math.abs(single_output)
    abs_max = tf.math.reduce_max(single_output_abs, axis=-1) + kb.epsilon()
    activatedIdx = tf.expand_dims(tf.greater(abs_max, 0.5), axis=-1)
    normalized_single_output = single_output / tf.expand_dims(abs_max, axis=-1)
    normalized_single_output_activated = tf.where(activatedIdx, normalized_single_output, 0)
    
    for feature in range(numFeatures):
        single_output_FeatureActivated = normalized_single_output_activated[:,:,feature]
        pcm = axs[int(feature % 3), int(feature / 3 + 1)].pcolormesh(single_output_FeatureActivated, cmap='RdBu_r')
        axs[int(feature % 3), int(feature / 3 + 1)].set_title('Feature : ' + str(feature))
        
        if feature % 3 == 0:
            fig.colorbar(pcm, ax=axs[:,int(feature / 3 + 1)], location='right', shrink=0.6)
        
    
    for i in range(3 - numFeatures%3):
        axs[-(i+1), -1].axis('off')
    
    fig.tight_layout()
    
    plt.savefig('savedImgs/' + str(class_) + '/' + str(epoch) + '.jpg')
    plt.close(fig)

In [103]:
class customLoss__(tf.keras.losses.Loss):
    
    def __init__(self, name="distanceLoss"):
        super().__init__(name=name)
        
    def __call__(self, outputs):
        #if type_ == 0:
        #print(outputs.shape)
        featureVectors = tf.zeros((1, model.numFeatures))
        for single_output in outputs:
            tf.autograph.experimental.set_loop_options(shape_invariants=[(featureVectors, tf.TensorShape([None, model.numFeatures]))])

            single_output_activated = tf.where(tf.greater(single_output, 0.5), 1, 0)
            
            
            featureVector = tf.zeros(shape=(1,))
            for feature in range(model.numFeatures):
                tf.autograph.experimental.set_loop_options(shape_invariants=[(featureVector, tf.TensorShape([None,]))])
                
                single_output_FeatureActivated = single_output_activated[:,:,feature]
                count = tf.expand_dims(tf.cast(tf.math.count_nonzero(single_output_FeatureActivated), float), axis=0)
                featureVector = tf.concat([featureVector, count], axis=0)
            featureVector = featureVector[1:]
            #print(featureVector.shape)
            
            featureVector = tf.expand_dims(featureVector, axis=0)
            featureVectors = tf.concat([featureVectors, featureVector], axis=0)
            
        
        featureVectors = featureVectors[1:,:]
        
        #print(featureVectors)
        max_ = tf.expand_dims(tf.reduce_max(featureVectors, axis=1), axis=-1)
        min_ = tf.expand_dims(tf.reduce_min(featureVectors, axis=1), axis=-1)
        #print(max_.shape)
        
        featureVectors = tf.subtract(featureVectors, min_) / tf.subtract(max_, min_)
        #print(featureVectors)
        
        #print('!!')
        #print(featureVectors)
        featureVector_mean = tf.math.reduce_mean(featureVectors, axis=0)
        
        #print(featureVector_mean)
        tmp = tf.math.subtract(featureVectors, tf.expand_dims(featureVector_mean, axis=0))
        distance2mean = tf.norm(tmp, axis=1)
        distanceSum = tf.math.reduce_sum(distance2mean)
        #print(distance2mean)
        #print(distanceSum)
        Loss = tf.math.exp(-distanceSum)
        
        return distanceSum

In [104]:
class customLoss(tf.keras.losses.Loss):
    
    def __init__(self, name="distanceLoss"):
        super().__init__(name=name)
        
    def __call__(self, outputs):
        #if type_ == 0:
        #print(outputs.shape)
        featureVectors = tf.zeros((1, model.numFeatures))
        for single_output in outputs:
            tf.autograph.experimental.set_loop_options(shape_invariants=[(featureVectors, tf.TensorShape([None, model.numFeatures]))])
            single_output_abs = tf.math.abs(single_output)
            abs_max = tf.math.reduce_max(single_output_abs, axis=-1) + kb.epsilon()
            activatedIdx = tf.expand_dims(tf.greater(abs_max, 0.5), axis=-1)
            normalized_single_output = single_output / tf.expand_dims(abs_max, axis=-1)
            normalized_single_output_activated = tf.where(activatedIdx, normalized_single_output, 0)
            
            featureVector = tf.zeros(shape=(1,))
            for feature in range(model.numFeatures):
                tf.autograph.experimental.set_loop_options(shape_invariants=[(featureVector, tf.TensorShape([None,]))])
                
                single_output_FeatureActivated = normalized_single_output_activated[:,:,feature]
                #print(single_output_FeatureActivated.shape)
                sum_ = tf.expand_dims(tf.math.reduce_sum(single_output_FeatureActivated), axis=0)
                
                featureVector = tf.concat([featureVector, sum_], axis=0)
            featureVector = featureVector[1:]
            #print(featureVector.shape)
            
            featureVector = tf.expand_dims(featureVector, axis=0)
            featureVectors = tf.concat([featureVectors, featureVector], axis=0)
            
        featureVectors = featureVectors[1:,:]
        #print(featureVectors)
        featureVectors_abs = tf.math.abs(featureVectors)
        abs_max = tf.math.reduce_max(featureVectors_abs, axis=-1) +  + kb.epsilon()
        #print(abs_max.shape)
        featureVectors = featureVectors / tf.expand_dims(abs_max, axis=-1)
        
        
        #print(featureVectors)
        
        
        featureVector_mean = tf.math.reduce_mean(featureVectors, axis=0)
        
        #print(featureVector_mean)
        #print('!!')
        
        tmp = tf.math.subtract(featureVectors, tf.expand_dims(featureVector_mean, axis=0))
        distance2mean = tf.norm(tmp, axis=1)
        distanceSum = tf.math.reduce_sum(distance2mean)
        Loss = tf.math.exp(-distanceSum)
            
        
        return Loss


In [105]:
tmp1 = tf.random.normal(shape=(2, 2, 3))
print(tmp1)
tmp2 = tf.random.normal(shape=(2, 2))

tmp2 = tf.greater(tmp2, 0.5)
print(tmp2)
tmp3 = tf.expand_dims(tmp2, axis=-1)
#print(tmp3)
tmp4 = tf.where(tmp3, tmp1, 0)
print(tmp4)

tf.Tensor(
[[[-1.3828938   0.43966636  0.2457787 ]
  [ 0.63606316 -0.6373113   0.8395073 ]]

 [[ 0.7769656  -0.44148183  0.97496396]
  [ 1.1209984   0.8038084  -0.6012371 ]]], shape=(2, 2, 3), dtype=float32)
tf.Tensor(
[[False False]
 [False  True]], shape=(2, 2), dtype=bool)
tf.Tensor(
[[[ 0.         0.         0.       ]
  [ 0.         0.         0.       ]]

 [[ 0.         0.         0.       ]
  [ 1.1209984  0.8038084 -0.6012371]]], shape=(2, 2, 3), dtype=float32)


In [106]:
tmp1 = tf.random.normal(shape=(2, 2, 3))
print(tmp1)
tmp2 = tf.math.abs(tmp1)
print(tmp2)
tmp3 = tf.math.reduce_max(tmp2, axis=-1)
print(tmp3)
tmp4 = tmp1 / tf.expand_dims(tmp3, axis=-1)
print(tmp4)


tf.Tensor(
[[[ 0.19576989  1.765268   -0.9999262 ]
  [-0.62681615 -0.44974372 -0.31097123]]

 [[ 0.2895611  -1.4253116  -0.8290877 ]
  [ 0.6379421   0.19654004  0.6517966 ]]], shape=(2, 2, 3), dtype=float32)
tf.Tensor(
[[[0.19576989 1.765268   0.9999262 ]
  [0.62681615 0.44974372 0.31097123]]

 [[0.2895611  1.4253116  0.8290877 ]
  [0.6379421  0.19654004 0.6517966 ]]], shape=(2, 2, 3), dtype=float32)
tf.Tensor(
[[1.765268   0.62681615]
 [1.4253116  0.6517966 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[[ 0.11090095  1.         -0.56644446]
  [-1.         -0.717505   -0.49611235]]

 [[ 0.20315635 -1.         -0.5816887 ]
  [ 0.97874415  0.30153587  1.        ]]], shape=(2, 2, 3), dtype=float32)


In [107]:
tmp = tf.zeros(shape=(1,))
print(tmp)

loss1 = customLoss()

for i in range(inputs_train.shape[0]):
    inputs = inputs_train[i,:,:,:,:]
    #print(inputs.shape)
    outputs = model(inputs)
    l1 = loss1(outputs)
    break

tf.Tensor([0.], shape=(1,), dtype=float32)


In [108]:
class MyModel(Model):
    def __init__(self, numFeatures=3):
        super(MyModel, self).__init__()
        self.numFeatures = numFeatures
        self.input_shape_ = None
        self.custom_layers = []
        
    def build(self, input_shape):
        self.input_shape_ = input_shape[1:]
        
        self.custom_layers.append(Conv2D(self.numFeatures, 3, padding="same", input_shape=self.input_shape_))
        self.pooling_ = MaxPool2D()
            
    def addLayer(self, numFeatures):
        self.numFeatures = numFeatures
        """
        shape = tf.convert_to_tensor(self.input_shape_).numpy()
        for i in range(len(self.custom_layers)):
            shape[0] = shape[0] - 2
            shape[1] = shape[1] - 2
         
        self.custom_layers.append(Conv2D(self.numFeatures, 3, input_shape=shape))
        """
        self.custom_layers.append(Conv2D(self.numFeatures, 3, padding="same", input_shape=self.input_shape_))
        self.custom_layers[-1].build(self.input_shape_)
        
            
    def addNode(self, numFeatures):
        weights = self.custom_layers[-1].get_weights()
        
        #print(weights[0].shape)
        #print(weights[1].shape)
        
        shape = list(weights[0].shape[:-1])
        shape.append(numFeatures)
        #print(shape)
        
        #shape = np.expand_dims(shape, axis=-1)
        tmp = np.random.normal(size=shape)
        weights[0] = np.concatenate((weights[0], tmp), axis=-1)
        #print(weights[0].shape)
        
        tmp = np.random.normal(size = (numFeatures,))
        weights[1] = np.concatenate((weights[1], tmp))
        
        """
        #print(weights[1].shape)
        shape = tf.convert_to_tensor(self.input_shape_).numpy()
        for i in range(len(self.custom_layers) - 1):
            shape[0] = shape[0] - 2
            shape[1] = shape[1] - 2
        """
        self.numFeatures += numFeatures
        self.custom_layers[-1] = Conv2D(self.numFeatures, 3, input_shape=self.input_shape_)
        self.custom_layers[-1].build(self.input_shape_)
        self.custom_layers[-1].set_weights(weights)
        
        #self.custom_layers[-1] = Conv2D(self.numFeatures, 3, input_shape=shape)
        #self.custom_layers[-1].build(shape)
        #self.custom_layers[-1].set_weights(weights)

    def call(self, x):
        
        for idx_layer, layer_ in enumerate(self.custom_layers):
            x = tf.math.sin(layer_(x))
            #x = self.pooling_(x)
        return x

model = MyModel(numFeatures=3)

In [109]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss1 = customLoss()
L1_train = tf.keras.metrics.Mean()
#L2_train = tf.keras.metrics.Mean()

@tf.function
def train_step(inputs):
    with tf.GradientTape() as tape:
        outputs = model(inputs)
        
        l1 = loss1(outputs)
        Loss = l1
        
    gradients = tape.gradient(Loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    L1_train(l1)
    
L1_valid = tf.keras.metrics.Mean()
#L2_valid = tf.keras.metrics.Mean()
@tf.function
def test_step(inputs):
    outputs = model(inputs)
    l1 = loss1(outputs)
    L1_valid(l1)

In [127]:
EPOCHS = 20000
patience = 300
stopped_epoch = 0
best_weights = None
best = np.Inf
wait = 0

history_l1_train = []
#history_l2_train = []
history_l1_valid = []
#history_l2_valid = []


#model.addNode(2)
#model.addLayer(20)
start_time = time.time()
for epoch in range(EPOCHS):
    clear_output(wait=True)
    
    L1_train.reset_states()
    #L2_train.reset_states()
    for i in range(inputs_train.shape[0]):
        inputs = inputs_train[i,:,:,:,:]
        train_step(inputs)
    
    L1_valid.reset_states()
    #L2_valid.reset_states()
    for i in range(inputs_test.shape[0]):
        inputs = inputs_test[i,:,:,:,:]
        test_step(inputs)
    
    template = '에포크: {}, L1_train: {:.4f}, L1_vaid: {:.4f}, 걸린 시간: {:.3f}'
    print (template.format(epoch+1,
                         L1_train.result(),
                         L1_valid.result(),
                           (time.time() - start_time)))
    
    if epoch % 50 == 0:
        for class_ in range(5):
            drawFeatureMap(epoch = epoch, model = model, idxBatch = 0, class_ = class_)
            

    """
    template = '에포크: {}, L1_train: {:.4f}, L2_train: {:.4f}, L1_vaid: {:.4f}, L2_valid: {:.4f}, 걸린 시간: {:.3f}'
    print (template.format(epoch+1,
                         L1_train.result(),
                           L2_train.result(),
                         L1_valid.result(),
                           L2_valid.result(),
                           (time.time() - start_time)))
    """
    
    history_l1_train.append(L1_train.result())
    #history_l2_train.append(L2_train.result())
    
    history_l1_valid.append(L1_valid.result())
    #history_l2_valid.append(L2_valid.result())
    
    """
    if np.less(float(L1_valid.result() + L2_valid.result()), best):
        best = float(L1_valid.result() + L2_valid.result())
        best_weights = model.get_weights()
        wait = 0
    """
    if np.less(float(L1_valid.result()), best):
        best = float(L1_valid.result())
        best_weights = model.get_weights()
        wait = 0
        
    else:
        wait +=1
        if wait >= patience:
            model.set_weights(best_weights)
            stopped_epoch = epoch
            print('Early Stopped !')
            break
    
    
fig, ax = plt.subplots(1, figsize=(18,20))

ax.set_title('L1')
x = np.arange(len(history_l1_train))
ax.plot(x, history_l1_train, 'r-', label = 'trainLoss')
ax.plot(x, history_l1_valid, 'b-', label = 'validLoss')
ax.legend()

"""
ax[1].set_title('L2')
ax[1].plot(x, history_l2_train, 'r-', label = 'trainLoss')
ax[1].plot(x, history_l2_valid, 'b-', label = 'validLoss')
ax[1].legend()
fig.tight_layout()
plt.show()
"""

KeyboardInterrupt: 

In [ ]:
model.summary()